In [1]:
import requests
#request помогает писать запрос к сайту и вытягивать оттуда данные, статистику

from bs4 import BeautifulSoup
#разбирает html таблицу,делает из нее объект,с которым можно работать

import csv 

In [15]:
#создаем константы
#сюда сохраним csv file
CSV='results.csv'

#передаем домен, с которого парсим
HOST='https://www.hltv.org/'

#вставляем URL с которого берем данные
URL ='https://www.hltv.org/results'

#Чтобы сайты не подумали,что мы боты, нужно передавать заголовки
#создаем словарь, в который передаем значения из кода страницы
HEADERS = {
    'accept': '*/*',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}

In [3]:
#создаем функции для парсинга
#обращение к страницы для получения html
#Параметры это кусочки url на новых страницах
def get_html(url,params=''):
    r=requests.get(url,headers=HEADERS,params=params)
    return r

In [4]:
html=get_html(URL)

In [5]:
print(html)

<Response [200]>


In [8]:
#получаем контент сайта из hlml
#в цикле наполняем пустой список необходимыми данными: команда 1, команда 2 , счет, организатор, тип матча
#ориентируемся на код страницы, указатели типа div, td и вытаскиваем нужные данные
def get_content(html):
    soup=BeautifulSoup(html,'html.parser')
    items=soup.find_all('div',class_="result-con")
    results=[]
    
    for item in items:
        results.append(
            {
                'team1': item.find('div',class_="line-align team1").get_text(strip=True),
                'score': str(item.find('td', class_="result-score").get_text()),
                'team2': item.find('div',class_="line-align team2").get_text(strip=True),
                'event': item.find('td', class_="event").get_text(),
                'format': item.find('td', class_="star-cell").get_text(strip=True)
            
            }
        )
    
    return results

In [9]:
print(get_content(html.text))

[{'team1': 'Budapest Five', 'score': '1 - 2', 'team2': 'Enterprise', 'event': 'V4 Future Sports Festival 2021 Play-In', 'format': 'bo3'}, {'team1': 'Finest', 'score': '2 - 0', 'team2': '00Nation', 'event': 'Pinnacle Fall Series 3 Regionals', 'format': 'bo3'}, {'team1': 'MASONIC', 'score': '2 - 0', 'team2': '4glory', 'event': 'Pinnacle Fall Series 3 Regionals', 'format': 'bo3'}, {'team1': 'Savage', 'score': '2 - 1', 'team2': 'KOVA', 'event': 'Elisa Invitational Winter 2021 Regionals', 'format': 'bo3'}, {'team1': 'GamerLegion', 'score': '2 - 0', 'team2': 'Lynn Vision', 'event': 'William Hill Cup 2021', 'format': 'bo3'}, {'team1': 'TYLOO', 'score': '2 - 1', 'team2': 'Run or Die', 'event': 'Funspark ULTI 2021 Europe Regional Series 4 Closed Qualifier', 'format': 'bo3'}, {'team1': 'K23', 'score': '0 - 2', 'team2': 'forZe', 'event': 'REPUBLEAGUE TIPOS Season 2', 'format': 'bo3'}, {'team1': 'INDE IRAE', 'score': '0 - 2', 'team2': 'Eternal Fire', 'event': 'Funspark ULTI 2021 Europe Regional Se

In [10]:
def save_doc(items,path):
    with open(path,'w',newline='') as file:
        writer=csv.writer(file,delimiter=';')
        writer.writerow(['Team 1','Result','Team 2','Event','Match Type'])
        for item in items:
            writer.writerow([item['team1'],item['score'],item['team2'],item['event'],item['format']])

In [16]:
#цикл внутри будет работать с каждой страницей (пагинация)
#в нашем случае сайта hltv тут вводим кол-во матчей
def parser():
    pagenation=int(input('Укажите кол-во матчей для парсинга: '))
    html=get_html(URL)
    if html.status_code==200:
        results=[]
        for game in range(0, pagenation+1):
            print(f'Парсим матчи :{game}')
            html=get_html(URL, params={'offset':game})
            results.extend(get_content(html.text))
            save_doc(results,CSV)
    else:
        print('Site is not available')

In [17]:
parser()

Укажите кол-во матчей для парсинга: 2
Парсим матчи :0
Парсим матчи :1
Парсим матчи :2
